<a href="https://colab.research.google.com/github/brucenielson/Genetic-Programming/blob/master/Genetic_Programming_Experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook contains the code I used for my inital Genetic Programming experiments. I used Toby Segaran's Genetic Programming code from his book [Collective Intelligence](https://www.amazon.com/gp/product/0596529325/ref=as_li_tl?ie=UTF8&camp=1789&creative=9325&creativeASIN=0596529325&linkCode=as2&tag=thelightrebor-20&linkId=6d3a6fa3b0f037c6a9fcee2b9e3355c8) as the starting point. ([Code found here](https://resources.oreilly.com/examples/9780596529321/tree/master/PCI_Code%20Folder/chapter11)) This experiment was primarily about playing with the hyper parameters of the code to get best results. However, I make a few improvements along the way to improve performance via various tricks I'll explain.

First we will load some stats functions that are useful for calculating standard deviations. I used the code found [here](https://stackoverflow.com/questions/15389768/standard-deviation-of-a-list). However, it says it's just the code taken staight from Python's statistic module.

In [0]:
# From: https://stackoverflow.com/questions/15389768/standard-deviation-of-a-list

def mean(data):
    """Return the sample arithmetic mean of data."""
    n = len(data)
    if n < 1:
        raise ValueError('mean requires at least one data point')
    return sum(data)/n # in Python 2 use sum(data)/float(n)

def _ss(data):
    """Return sum of square deviations of sequence data."""
    c = mean(data)
    ss = sum((x-c)**2 for x in data)
    return ss

def stddev(data, ddof=1):
    """Calculates the population standard deviation
    if ddof=0; specify ddof=1 to compute the sample
    standard deviation. (which I made default)"""
    n = len(data)
    if n < 2:
        raise ValueError('variance requires at least two data points')
    ss = _ss(data)
    pvar = ss/(n-ddof)
    return pvar**0.5

Now load the function wrapper code and list of functions we'll be using in our genetic programming program trees: Add, Multiply, If, >, Subtract.

In [0]:
class fwrapper:
    def __init__(self, function, params, name):
        self.function = function
        self.params = params
        self.name = name


# Functions with 2 parameters
addw = fwrapper(lambda p: p[0] + p[1], 2, 'add')
subw = fwrapper(lambda p: p[0] - p[1], 2, 'subtract')
mulw = fwrapper(lambda p: p[0] * p[1], 2, 'multiply')
# If and > Function
def iffunc(l):
    if l[0] > 0:
        return l[1]
    else:
        return l[2]

ifw = fwrapper(iffunc, 3, 'if')


def isgreater(l):
    if l[0] > l[1]:
        return 1
    else:
        return 0
gtw = fwrapper(isgreater, 2, 'isgreater')

# List of possible functions
flist = [addw, mulw, ifw, gtw, subw]


Required imports:

In [0]:
from random import random, randint, choice
from copy import deepcopy
from math import log
import datetime
import operator

Node classes: "node" is for functions, "paramnode" is for parameters, "constnode" is for constants

In [0]:
class node:
    treecounter = 0
    def __init__(self, fw, children):
        self.function = fw.function
        self.name = fw.name
        self.children = children
        self.id = node.treecounter
        self.lock = False
        node.treecounter += 1 # Sets a unique id on each node for tracking


    def evaluate(self, inp):
        results = [n.evaluate(inp) for n in self.children]
        return self.function(results)

    def display(self, indent=0):
        if self.lock:
            add = "*"
        else:
            add = ""
        print (' ' * indent) + self.name + add
        for c in self.children:
            c.display(indent + 1)


class paramnode:
    def __init__(self, idx):
        self.idx = idx

    def evaluate(self, inp):
        return inp[self.idx]

    def display(self, indent=0):
        print '%sp%d' % (' ' * indent, self.idx)


class constnode:
    def __init__(self, value):
        self.value = value

    def evaluate(self, inp):
        return self.value

    def display(self, indent=0):
        print '%s%d' % (' ' * indent, self.value)



Now we'll create an example programming tree manually to try this out. This tree is equivalent to:

```
def func(x,y):
  if x>3:
    return y + 5
  else:
    return y - 2
```



In [0]:
class exampletree(node):
    def __init__(self):
        children = [node(gtw, [paramnode(0), constnode(3)]),
                    node(addw, [paramnode(1), constnode(5)]),
                    node(subw, [paramnode(1), constnode(2)])]

        node.__init__(self, ifw, children)




In [0]:
exampletree = exampletree()

In [7]:
exampletree.evaluate([2,3])

1

In [8]:
exampletree.evaluate([5,3])

8

We can also display the tree in a (somewhat) more readable format by calling the "display" method

In [9]:
exampletree.display()

if
 isgreater
  p0
  3
 add
  p1
  5
 subtract
  p1
  2


Now let's create a way to make random code trees.

In [0]:
def makerandomtree(pc, maxdepth=4, fpr=0.5, ppr=0.6): # pc is number of parameters
    if random() < fpr and maxdepth > 0:
        f = choice(flist)
        children = [makerandomtree(pc, maxdepth - 1, fpr, ppr)
                    for i in range(f.params)]
        return node(f, children)
    elif random() < ppr:
        return paramnode(randint(0, pc - 1))
    else:
        return constnode(randint(0, 10))


In [11]:
random1 = makerandomtree(2)
random1.display()

if
 p0
 subtract
  p1
  p0
 if
  p1
  10
  isgreater
   p0
   subtract
    4
    5


In [12]:
random1.evaluate([5,2])

-3

Now for the real experiments! We need a 'hidden function' that our genetic programming tries to figure out what it is. This function is equivalent to X^2 + 2Y + 3X + 5

In [0]:
def hiddenfunction(x, y):
    return x ** 2 + 2 * y + 3 * x + 5


Now create a random set of parameters and get the results. This will be the 'observations' that the genetic program will run against to determine how close it's program is to the correct one.

In [0]:
def buildhiddenset():
    rows = []
    for i in range(200):
        x = randint(0, 40)
        y = randint(0, 40)
        rows.append([x, y, hiddenfunction(x, y)])
    return rows



In [0]:
hiddenset = buildhiddenset()

In [16]:
print hiddenset

[[25, 34, 773], [14, 9, 261], [6, 38, 135], [22, 23, 601], [38, 35, 1633], [7, 16, 107], [37, 38, 1561], [34, 15, 1293], [29, 31, 995], [8, 3, 99], [14, 18, 279], [20, 15, 495], [26, 17, 793], [20, 38, 541], [21, 15, 539], [2, 12, 39], [3, 4, 31], [5, 39, 123], [15, 37, 349], [38, 2, 1567], [23, 32, 667], [33, 32, 1257], [3, 19, 61], [2, 14, 43], [27, 16, 847], [25, 35, 775], [25, 29, 763], [39, 20, 1683], [21, 11, 531], [9, 30, 173], [35, 4, 1343], [34, 16, 1295], [36, 12, 1433], [28, 2, 877], [12, 40, 265], [3, 39, 101], [30, 23, 1041], [26, 12, 783], [36, 13, 1435], [11, 25, 209], [20, 6, 477], [36, 9, 1427], [38, 18, 1599], [17, 13, 371], [28, 12, 897], [27, 1, 817], [1, 22, 53], [33, 24, 1241], [8, 17, 127], [31, 26, 1111], [22, 30, 615], [17, 11, 367], [28, 0, 873], [17, 32, 409], [38, 2, 1567], [26, 26, 811], [25, 15, 735], [6, 38, 135], [9, 28, 169], [17, 38, 421], [24, 0, 653], [23, 8, 619], [6, 18, 95], [2, 32, 79], [24, 6, 665], [9, 36, 185], [9, 7, 127], [17, 9, 363], [16, 

Let's create a 'score' function that evaluates a genetic programming tree and determines how far off it's function is from the correct result.

In [0]:
def scorefunction(tree, dataset, penalizecomplexity=False):
    start = datetime.datetime.now()
    dif = 0
    for row in dataset:
        val = tree.evaluate([row[0], row[1]])
        dif += abs(val - row[2])

    if penalizecomplexity == False: # We'll use this code later to penalize more complext trees
        return (dif, tree)
    else:
        end = datetime.datetime.now()
        delta = end - start
        seconds = delta.total_seconds()
        return (dif, tree, seconds)


In [18]:
scorefunction(random1, hiddenset)[0]

131335

Now we'll create the mutate and crossover functions. Mutate takes a tree and has a chance of changing one node into a new random program. Crossover takes two trees and has a chance of grabbing a node of one tree and splicing it into the other. 

In [0]:
def mutate(t, pc, probchange=0.1):
    if random() < probchange:
        return makerandomtree(pc)
    else:
        result = deepcopy(t)
        if hasattr(t, "children"):
            result.children = [mutate(c, pc, probchange) for c in t.children]

        return result


def crossover(t1, t2, probswap=0.1, top=True):
    # print "t1:", getattr(t1, "id", -1), "t2:", getattr(t2, "id", -1)
    if random() < probswap and not top:
        # print "return t2:", getattr(t2, "id", -1)
        return deepcopy(t2)
    else:
        result = deepcopy(t1)
        if hasattr(t1, 'children') and hasattr(t2, 'children') and not getattr(result, "lock", False):
            result.children = [crossover(c, choice(t2.children), probswap, top=False)
                               for c in t1.children]
        # print "return crossover:", getattr(result, "id", -1)
        return result


In [20]:
random2 = makerandomtree(2)
random2.display()

if
 p1
 isgreater
  p1
  p0
 5


In [21]:
mutatedtree = mutate(random2, 2)
mutatedtree.display()

subtract
 isgreater
  if
   p1
   p1
   p0
  p0
 9


In [22]:
crossovertree = crossover(random1, random2)
crossovertree.display()

if
 p0
 subtract
  p1
  p0
 if
  p1
  10
  isgreater
   p0
   subtract
    4
    5


Finally, we're going to create an evolve function that creates a set of randomtrees, then evaluates them against the hidden function. It sorts them into order of which are best, then starts to 'evolve' them via mutation and crossover.

def evolve(pc, popsize, rankfunction, maxgen=500,
         mutationrate=0.1, breedingrate=0.3, fitnesspref=0.7, probnew=0.05,
         penalizecomplexity=False, doublemutate=True)

In [0]:
def evolve(pc, popsize, rankfunction, maxgen=500,
         mutationrate=0.2, breedingrate=0.1, fitnesspref=0.7, probnew=0.1,
         detectstuck=False, mute=False):
  # Returns a random number, tending towards lower numbers. The lower pexp
  # is, more lower numbers you will get
  def selectindex():
      return int(log(random()) / log(fitnesspref))

  # Create a random initial population
  population = [makerandomtree(pc) for i in range(popsize)]
  lastscore = None
  stuckcounter = 0
  for i in range(maxgen):
      scores = rankfunction(population)

      # If we get same value too often, take action
      adj_mutate = mutationrate

      if not mute:
        print "Generation:", i+1, "Best Score:", scores[0][0]
          
      if scores[0][0] == 0: break

      # The two best always make it
      newpop = [scores[0][1], scores[1][1]]

      # Build the next generation
      while len(newpop) < popsize:
          if random() > probnew:
              newpop.append(mutate(
                  crossover(scores[selectindex()][1],
                            scores[selectindex()][1],
                            probswap=breedingrate),
                  pc, probchange=adj_mutate))
          else:
              # Add a random node to mix things up
              newpop.append(makerandomtree(pc))

      population = newpop
  
  if not mute:
    print "******"
    print "Best Tree Found:"
    scores[0][1].display()
    
  return (scores, i+1)


We also need a rankfunction that scores and sorts the population of programs.

In [0]:
def getrankfunction(dataset):
    def rankfunction(population):
        scores = [scorefunction(t, dataset) for t in population]
        scores.sort()

        return scores

    return rankfunction



Ready to run and see if our genetic programming population can find the real hidden function. My experience is that it finds it a bit less than half the time with the default parameters.

In [0]:
rf = getrankfunction(hiddenset)

In [26]:
results = evolve(2, 500, rf, maxgen=50, mutationrate=0.2, breedingrate=0.1, fitnesspref=0.7, probnew=0.1)

Generation: 1 Best Score: 15661
Generation: 2 Best Score: 400
Generation: 3 Best Score: 200
Generation: 4 Best Score: 200
Generation: 5 Best Score: 200
Generation: 6 Best Score: 3
Generation: 7 Best Score: 0
******
Best Tree Found:
add
 p0
 add
  add
   add
    p0
    add
     add
      p0
      subtract
       9
       4
     p1
   multiply
    p0
    p0
  p1


At last, we're ready to start experimenting with hyper parameters to find the best ones! 

So the first thing we're going to try is playing with mutationrate, breedingrate, fitnesspref (preference for how bias we should grab the most fit trees to breed), and probnew (probability of creating a new random tree).

To do this experiment, we need a test function that will run evolve multiple times and give us averages and standard deviations across mulitple runs. 

But there is so much deviation from one run to the next we really should run 1000 different times and then take the average result. But we can't wait that long, so we'll do 5 instead. 

Consider changing that to 50 and you'll get a fairly good feel for averages using different hyper parameters within an hour or two.

In [0]:
def getstats(rounds=50, maxgen=50, mutationrate=0.05, breedingrate=0.10, fitnesspref=0.95, probnew=0.10, penalizecomplexity=False, modularize=False, mute=False):
    dataset = buildhiddenset()
    rf = getrankfunction(dataset)
    tries = []
    for i in range(rounds):
        if not mute:
          print "*******Round: ", i+1, "*******"
        start = datetime.datetime.now()
        scores, generations = evolve(2, 500, rf, maxgen=maxgen, mutationrate=mutationrate, breedingrate=breedingrate, fitnesspref=fitnesspref, probnew=probnew,  mute=mute)
        best = scores[0][1]
        score = scorefunction(best, dataset)
        end = datetime.datetime.now()
        delta = end - start
        seconds = delta.total_seconds()
        row = (score[0], seconds, generations, best)
        tries.append(row)

    scores = [row[0] for row in tries]
    avg_score = sum(scores) / len(scores)
    std_score = stddev(scores)

    successes = scores.count(0)

    times = [row[1] for row in tries]
    avg_time = sum(times) / len(times)
    std_time = stddev(times)

    generations = [row[2] for row in tries]
    avg_generations = sum(generations) / len(generations)
    std_generations = stddev(generations)

    # print "Final Population", getids(population)
    print "# of Successes:", successes
    print "Average Score:", avg_score, "StD:", round(std_score, 2)
    print "Average Time (Seconds):", avg_time, "StD:", round(std_time, 2)
    print "Average Generations:", avg_generations, "StD:", round(std_generations, 2)

    return successes, avg_score, avg_time, avg_generations



In [28]:
getstats(rounds=5, maxgen=50, mutationrate=0.2, breedingrate=0.1, fitnesspref=0.7, probnew=0.1, mute=True)

# of Successes: 2
Average Score: 1758 StD: 1681.93
Average Time (Seconds): 119.080995 StD: 31.3
Average Generations: 50 StD: 0.0


(2, 1758, 119.080995, 50)

So the above is the default settings. Can we do better? Playing around with the hyper parameters, the follows was the best I could find. I think there is something to be learned from these:

*   **Mutation Rate**: A very low mutation rate of 0.05 worked the best. The obvious advantage here is that it more slowly makes changes and doesn't produce garbage too fast. The downside is that we're giving up an opportunity for more variation to select from generation to generation. I think there is an added bonus of a low mutation rate: a quirk in the code shows that a higher mutation rate will tend to force the mutations to the top because it starts at the top and only keeps moving down if no mutation takes place in the parents. This suggests that a lower mutation rate allows more mutations lower into the tree. 
*   **Breeding Rate**: A slightly higher breeding rate of 0.10 seemed to work the best. This suggests that crossover is probably the main type of variation (compared to mutations)
*   **Fitness Preference**: Fitness Preference determines the liklihood of selecting less fit trees to crossbreed with. The value should be less than 1.0. I experimented and found that the best value here is 0.95, which is fairly high. This makes good sense given the low breeding rate because it injects more variation into the population, even though it is less fit variation. This shows the importance of variation.
*   **Probability New**: The probability of, instead of crossbreeding and mutating from existing trees in the population that we just build a new one from scratch. This is necessary to keep injecting new 'genetic material' into the population. This is another source of variation.





In [29]:
getstats(rounds=5, maxgen=50, mutationrate=0.05, breedingrate=0.10, fitnesspref=0.95, probnew=0.10, mute=True)

# of Successes: 3
Average Score: 486 StD: 983.74
Average Time (Seconds): 199.1788338 StD: 243.3
Average Generations: 29 StD: 19.42


(3, 486, 199.1788338, 29)

Having found the best hyper parameters, I wanted to see if I could improve things by giving a penalty for more complex trees and thereby emphasize trees that gave good results, but were less complex. 

It's not immediately obvious how to do this. The way the code is currently written, I'd have to traverse the whole tree to calculate it's complexity. So I cheated and just kept track of how long the tree takes to run. This isn't perfect, but it should give me a rough order of trees that score the same but some run faster than others (presumably due to reduced complexity from less 'junk code')

So I rewrote the evolve function and score function to track how long it takes to evaluate a tree and to order first by score and then by the length of time it took to run the tree. 

In [0]:
def getstats(rounds=50, maxgen=50, mutationrate=0.05, breedingrate=0.10, fitnesspref=0.95, probnew=0.10, penalizecomplexity=False, modularize=False, mute=False):
    dataset = buildhiddenset()
    rf = getrankfunction(dataset)
    tries = []
    for i in range(rounds):
        if not mute:
          print "*******Round: ", i+1, "*******"
        start = datetime.datetime.now()
        scores, generations = evolve(2, 500, rf, maxgen=maxgen, mutationrate=mutationrate, breedingrate=breedingrate, fitnesspref=fitnesspref, probnew=probnew,  penalizecomplexity=penalizecomplexity, mute=mute)
        best = scores[0][1]
        score = scorefunction(best, dataset)
        end = datetime.datetime.now()
        delta = end - start
        seconds = delta.total_seconds()
        row = (score[0], seconds, generations, best)
        tries.append(row)

    scores = [row[0] for row in tries]
    avg_score = sum(scores) / len(scores)
    std_score = stddev(scores)

    successes = scores.count(0)

    times = [row[1] for row in tries]
    avg_time = sum(times) / len(times)
    std_time = stddev(times)

    generations = [row[2] for row in tries]
    avg_generations = sum(generations) / len(generations)
    std_generations = stddev(generations)

    # print "Final Population", getids(population)
    print "# of Successes:", successes
    print "Average Score:", avg_score, "StD:", round(std_score, 2)
    print "Average Time (Seconds):", avg_time, "StD:", round(std_time, 2)
    print "Average Generations:", avg_generations, "StD:", round(std_generations, 2)

    return successes, avg_score, avg_time, avg_generations



In [0]:
def getrankfunction(dataset):
    def rankfunction(population, penalizecomplexity=False):
        scores = [scorefunction(t, dataset, penalizecomplexity) for t in population]
        if penalizecomplexity:
            scores = sorted(scores, key=operator.itemgetter(0, 2))
        else:
            scores.sort()

        return scores

    return rankfunction


  
def evolve(pc, popsize, rankfunction, maxgen=500,
         mutationrate=0.2, breedingrate=0.1, fitnesspref=0.7, probnew=0.1,
         penalizecomplexity=False, detectstuck=False, mute=False):
  # Returns a random number, tending towards lower numbers. The lower pexp
  # is, more lower numbers you will get
  def selectindex():
      return int(log(random()) / log(fitnesspref))

  # Create a random initial population
  population = [makerandomtree(pc) for i in range(popsize)]
  lastscore = None
  stuckcounter = 0
  for i in range(maxgen):
      scores = rankfunction(population, penalizecomplexity)

      # If we get same value too often, take action
      adj_mutate = mutationrate
      if detectstuck:
          if scores[0][2] == lastscore:
              stuckcounter += 1
          else:
              stuckcounter = 0

          lastscore = scores[0][2]

          if stuckcounter > 0:
              adj_mutate = mutationrate + 2.0*(float(stuckcounter)/100.0)
              if adj_mutate > 0.5: adj_mutate = 0.5

      if not mute:
        if penalizecomplexity:
            print "Generation:", i+1, "Best Score:", scores[0][0], "Time:", scores[0][2]
        else:
            print "Generation:", i+1, "Best Score:", scores[0][0]
          
      if scores[0][0] == 0: break

      # The two best always make it
      newpop = [scores[0][1], scores[1][1]]

      # Build the next generation
      while len(newpop) < popsize:
          if random() > probnew:
              newpop.append(mutate(
                  crossover(scores[selectindex()][1],
                            scores[selectindex()][1],
                            probswap=breedingrate),
                  pc, probchange=adj_mutate))
          else:
              # Add a random node to mix things up
              newpop.append(makerandomtree(pc))

      population = newpop
  
  if not mute:
    print "******"
    print "Best Tree Found:"
    scores[0][1].display()
    
  return (scores, i+1)


In [32]:
getstats(rounds=5, maxgen=50, mutationrate=0.05, breedingrate=0.10, fitnesspref=0.95, probnew=0.10, mute=True, penalizecomplexity=True)

# of Successes: 2
Average Score: 1505 StD: 1403.62
Average Time (Seconds): 115.3971174 StD: 100.16
Average Generations: 37 StD: 17.55


(2, 1505, 115.3971174, 37)

This change to take speed/complexity into consideration seems to improve things quite a bit. The code runs faster too, but smaller trees seem to have better fitness overall compared to other trees with seemingly equivalent scores but that run slower. 

Now for a final experiment. One problem in genetic programming is that useful code gets regularly destroyed by the crossover and mutation process. So I wondered what would happen if I 'locked' some sub-trees and let them breed into the population. This revised version of the evolve function now has a 'modularize' option that will take the best tree and copy it and 'lock' (with some probability) some of the sub-trees and then let it breed into the population. 

In [0]:
def getstats(rounds=50, maxgen=50, mutationrate=0.05, breedingrate=0.10, fitnesspref=0.95, probnew=0.10, penalizecomplexity=False, modularize=False, mute=False):
    dataset = buildhiddenset()
    rf = getrankfunction(dataset)
    tries = []
    for i in range(rounds):
        if not mute:
          print "*******Round: ", i+1, "*******"
        start = datetime.datetime.now()
        scores, generations = evolve(2, 500, rf, maxgen=maxgen, mutationrate=mutationrate, breedingrate=breedingrate, fitnesspref=fitnesspref, probnew=probnew, penalizecomplexity=penalizecomplexity, modularize=modularize, mute=mute)
        best = scores[0][1]
        score = scorefunction(best, dataset)
        end = datetime.datetime.now()
        delta = end - start
        seconds = delta.total_seconds()
        row = (score[0], seconds, generations, best)
        tries.append(row)

    scores = [row[0] for row in tries]
    avg_score = sum(scores) / len(scores)
    std_score = stddev(scores)

    successes = scores.count(0)

    times = [row[1] for row in tries]
    avg_time = sum(times) / len(times)
    std_time = stddev(times)

    generations = [row[2] for row in tries]
    avg_generations = sum(generations) / len(generations)
    std_generations = stddev(generations)

    # print "Final Population", getids(population)
    print "# of Successes:", successes
    print "Average Score:", avg_score, "StD:", round(std_score, 2)
    print "Average Time (Seconds):", avg_time, "StD:", round(std_time, 2)
    print "Average Generations:", avg_generations, "StD:", round(std_generations, 2)

    return successes, avg_score, avg_time, avg_generations

In [0]:
# This is the locksubtree function that locks subtrees
def locksubtree(t, probchange=0.05, top=True):
    result = deepcopy(t)

    if not hasattr(result, "children"):
        return result
    if result.lock:
        return result
    if random() < probchange and not top:
        result.lock = True
        return result
    else:
        result.children = [locksubtree(c, probchange, top=False) for c in result.children]
        return result


In [0]:
def getrankfunction(dataset):
    def rankfunction(population, penalizecomplexity=False):
        scores = [scorefunction(t, dataset, penalizecomplexity) for t in population]
        if penalizecomplexity:
            scores = sorted(scores, key=operator.itemgetter(0, 2))
        else:
            scores.sort()

        return scores

    return rankfunction


  
def evolve(pc, popsize, rankfunction, maxgen=500,
         mutationrate=0.2, breedingrate=0.1, fitnesspref=0.7, probnew=0.1,
         penalizecomplexity=False, detectstuck=False, modularize=False, mute=False):
  # Returns a random number, tending towards lower numbers. The lower pexp
  # is, more lower numbers you will get
  def selectindex():
      return int(log(random()) / log(fitnesspref))

  # Create a random initial population
  population = [makerandomtree(pc) for i in range(popsize)]
  lastscore = None
  stuckcounter = 0
  for i in range(maxgen):
      scores = rankfunction(population, penalizecomplexity)

      # If we get same value too often, take action
      adj_mutate = mutationrate
      if detectstuck:
          if scores[0][2] == lastscore:
              stuckcounter += 1
          else:
              stuckcounter = 0

          lastscore = scores[0][2]

          if stuckcounter > 0:
              adj_mutate = mutationrate + 2.0*(float(stuckcounter)/100.0)
              if adj_mutate > 0.5: adj_mutate = 0.5

      if not mute:
        if penalizecomplexity:
            print "Generation:", i+1, "Best Score:", scores[0][0], "Time:", scores[0][2]
        else:
            print "Generation:", i+1, "Best Score:", scores[0][0]
          
      if scores[0][0] == 0: break

      # The two best always make it
      newpop = [scores[0][1], scores[1][1]]

      if modularize:
        # Next one is same as first one, but modularized
        newpop.append(locksubtree(newpop[0]))

      # Build the next generation
      while len(newpop) < popsize:
          if random() > probnew:
              newpop.append(mutate(
                  crossover(scores[selectindex()][1],
                            scores[selectindex()][1],
                            probswap=breedingrate),
                  pc, probchange=adj_mutate))
          else:
              # Add a random node to mix things up
              newpop.append(makerandomtree(pc))

      population = newpop
  
  if not mute:
    print "******"
    print "Best Tree Found:"
    scores[0][1].display()
    
  return (scores, i+1)


In [36]:
getstats(rounds=5, maxgen=50, mutationrate=0.05, breedingrate=0.10, fitnesspref=0.95, probnew=0.10, modularize = True, mute=True, penalizecomplexity=True)

# of Successes: 1
Average Score: 1484 StD: 1287.2
Average Time (Seconds): 257.8309846 StD: 99.62
Average Generations: 45 StD: 9.43


(1, 1484, 257.83098459999997, 45)